In [19]:
# Import libraries
import itertools
import string
import shutil
import time
start_time = time.time()

In [20]:
# Input and output file paths
LATIN_FILES_BASEPATH = "Input/2_gram_input/"
LEMMATIZER_PATH = "new_lemmatizer.csv"
NO_OF_FILES = 1
OUTPUT_FILE = "Output/2_gram_output_"+str(NO_OF_FILES)

In [21]:
# Cycle through files in path and create master RDD
basepath = LATIN_FILES_BASEPATH
fileList = []
for f in os.listdir(basepath):
    fileList.append(f)
fileList = sorted(fileList)

textFile = sc.textFile(basepath + "/" + fileList[0])
for f1 in fileList[1:NO_OF_FILES]:
    textFile = textFile.union(sc.textFile(basepath + "/" + f1))


In [22]:
lemmaDict = {}

In [23]:
# Create a dictionary of lemmas for lookup

def readLemmas():
    with open(LEMMATIZER_PATH) as f1:
        for line in f1.readlines():
            lemmas = line.split(',')
            lemmaDict[lemmas[0]] = list()
            for word in lemmas[1:]:
                if word=='':
                    break
                lemmaDict[lemmas[0]].append(word)

In [24]:
readLemmas()

In [25]:
# Preprocessing of lines to substitute lemmas and generate combinations
def checkWord(line):
    if (len(line)>0):
        position = line.split("> ")[0]+">"
        words=line.split("> ")[1].lower().translate(str.maketrans('','',string.punctuation)).split()
        fullList =[]
        for i in itertools.combinations(words,2):
            a = list(i)
            word1=i[0]
            word2=i[1]
            l1 =[]
            if word1 in lemmaDict:
                l1 = lemmaDict.get(word1)
            else:
                l1=[word1]
            if word2 in lemmaDict:
                l2 = lemmaDict.get(word2)
            else:
                l2=[word2]
            for lemma_comb in list(itertools.product(l1,l2)):
                fullList.append((position,tuple(sorted(lemma_comb)),))
        return fullList

In [26]:
lines = textFile.map(checkWord)

In [27]:
#lines.collect()

In [28]:
keyValues = lines.flatMap(lambda line:line)

In [29]:
#keyValues.collect()

In [30]:
# Return key as pair and value as postion
def processTuple(tuple1):
    return tuple1[1],[tuple1[0]]

In [31]:
mapper = keyValues.map(processTuple)

In [32]:
#mapper.collect()

In [33]:
# Combine positions for a pair
reducer = mapper.reduceByKey(lambda a,b: a+b)

In [34]:
#reducer.collect()

In [35]:
if os.path.exists(OUTPUT_FILE):
    shutil.rmtree(OUTPUT_FILE)
reducer.saveAsTextFile(OUTPUT_FILE)

In [36]:
print("--- %s seconds ---" % round(time.time() - start_time, 2))

--- 9.43 seconds ---
